Import common packages

In [1]:
import sys
import os
import datetime
import errno

Import findspark and import&init spark

In [2]:
import findspark


os.environ["SPARK_HOME"] = "/home/ser/Dev/Spark/spark-2.1.0-bin-hadoop2.7"
os.environ["PYSPARK_PYTHON"] = "/home/ser/Dev/Python/2/anaconda2/bin/python"
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[4] pyspark-shell"
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--master yarn \
                                     #--deploy-mode cluster \
                                     #--conf \"spark.yarn.am.port\"=8025\
                                     #pyspark-shell"
print os.environ['PYSPARK_SUBMIT_ARGS']
#os.environ['HADOOP_CONF_DIR'] = "/home/ser/Dev/Hadoop/hadoop-2.7.3/etc/hadoop"
findspark.init()

#import numpy as np
#import sep
#from operator import add

import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row

--master local[4] pyspark-shell


Import task-oriented packages

In [3]:
import astromatic_wrapper as aw
from astropy.io import fits
#import pydoop

Constants (in future go to config file)

In [4]:
exec_mode = 'test'
wrk_path = ''

In [5]:
sys.path.insert(0, '/home/ser/Dev/astro_engine/code')
from astro_utils import AEDirsTreeConfigurer

In [6]:
cfg = AEDirsTreeConfigurer(False, images_path='images')
cfg.build_dirs_tree()
cfg.new_log_dir()
cfg.get_paths()

{'catalogs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/catalogs',
 'config': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config',
 'images': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images',
 'logs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/logs',
 'stacks': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/stacks',
 'temp': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'}

In [7]:
class AEConfig:
    def __init__(self):
        self._scamp_required_sex_params = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 
                                           'AWIN_IMAGE', 'BWIN_IMAGE', 'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 
                                           'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD','ERRTHETA_WORLD', 
                                           'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO', 
                                           'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                                           # flags not supported yet (need Weightwatcher)
                                           #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                                           'FLAGS', 'FLUX_RADIUS', 'ELONGATION']

Spark code separately yet

In [45]:
def extract_1(input_item, broadcast_fits):
    from astro_utils import AEDirsTreeConfigurer
    cfg = AEDirsTreeConfigurer(False, images_path='images' + str(input_item[0][-1]))
    cfg.build_dirs_tree()
    cfg.new_log_dir()
    file_name = input_item[0].replace('file:', '')
    catalog_name = os.path.join(cfg.get_paths()['catalogs'], os.path.basename(file_name.replace('.fit', '.cat')))
    sex_kwargs_1 = {'code': 'SExtractor'}
    sex_kwargs_1['config_file'] = os.path.join(cfg.get_paths()['config'], 'default.sex')
    sex_kwargs_1['config'] = {'CATALOG_NAME': catalog_name}
    sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
    sex_kwargs_1['config']['FILTER'] = 'N'
    sex_kwargs_1['temp_path'] = cfg.get_paths()['temp']
    sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                              'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD', 
                              'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                              'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                              'FLAGS', 'FLUX_RADIUS', 'ELONGATION']
    sextractor = aw.api.Astromatic(**sex_kwargs_1)
    sextractor.run(file_name)
    #with open(catalog_name, "r") as catalog:
        #output_item = ('file:' + catalog_name, catalog)
    return catalog_name + str(broadcast_fits.value[0].header['BITPIX'])

In [8]:
def extract_2(input_item):
    file_name = input_item[0]
    return file_name

In [7]:
def extract_3(input_item):
    file_name = os.path.basename(input_item[0])
    tmp_file_name = os.path.join(cfg.get_paths()['temp'], file_name)
    catalog_name = os.path.join(cfg.get_paths()['catalogs'], file_name.replace('.fit', '.cat'))
    with open(tmp_file_name, "w+") as fits_file:
        fits_file.write(input_item[1])
    sex_kwargs_1 = {'code': 'SExtractor'}
    sex_kwargs_1['config_file'] = os.path.join(cfg.get_paths()['config'], 'default.sex')
    sex_kwargs_1['config'] = {'CATALOG_NAME': catalog_name}
    sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
    sex_kwargs_1['config']['FILTER'] = 'N'
    sex_kwargs_1['temp_path'] = cfg.get_paths()['temp']
    sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                              'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD', 
                              'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                              'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                              'FLAGS', 'FLUX_RADIUS', 'ELONGATION']
    sextractor = aw.api.Astromatic(**sex_kwargs_1)
    sextractor.run(tmp_file_name)
    with open(catalog_name, "r") as catalog:
        output_item = ('file:' + catalog_name, catalog)
    return output_item

In [81]:
def rt_1(input_i):
    return [a[0] for a in input_i]

In [ ]:
# TODO: check header_edit_script!!!

In [30]:
from functools import partial
from pyspark import SparkFiles

In [98]:
if __name__ == "__main__":
    sc = SparkContext(appName="SourceExtractor")
    sc.addPyFile('/home/ser/Dev/astro_engine/code/astro_utils.py')
    sc.addPyFile('/home/ser/Dev/astro_engine/code/fits_utils.py')
    bd_fits = fits.open('im_1.fit')
    broadcast_fits = sc.broadcast(bd_fits)
    
    files = sc.binaryFiles(cfg.get_paths()['images'])
    #files = sc.binaryFiles("hdfs://localhost:9000/ser/images/images")
    print files
    print broadcast_fits.value[0].header['RA']

org.apache.spark.api.java.JavaPairRDD@79d52e70
11 32 35.230


In [99]:
files = files.filter(lambda x: x[0].endswith('.fit'))

In [100]:
#print files.map(extract).reduce(join_cats)
#files.map(partial(extract_1, broadcast_fits=broadcast_fits)).map(lambda x: [x]).reduce(lambda a, b: a + b)
files.mapPartitions(rt_1).map(lambda x : [x]).reduce(lambda a, b: a + b)
#catalogs = files.map(extract_3).collect()
#len(catalogs)
#catalogs.map(lambda x: [x]).reduce(lambda a, b: a + b)
#catalogs.saveAsSequenceFile(cfg.get_paths()['catalogs'])

[u'file:/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_001.fit',
 u'file:/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_003.fit',
 u'file:/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_006.fit',
 u'file:/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_004.fit',
 u'file:/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_005.fit',
 u'file:/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_002.fit']

In [29]:
#catalogs[0]

(u'file:/home/ser/Dev/Notebooks/spark_pipeline_1/catalogs/GRB130427_R60_001_001.cat',
 <closed file '<uninitialized file>', mode '<uninitialized file>' at 0x7f573ba3e6f0>)

In [101]:
sc.stop()